<a href="https://colab.research.google.com/github/CristianoDataScience/project_microbusiness/blob/main/project_microbusiness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Arquivos**
Uma grande quantidade de dados está disponível publicamente sobre os condados e não tentamos reunir todos aqui. Você é fortemente encorajado a usar fontes de dados externas para recursos.

**trem.csv**

* row_id- Um código de ID para a linha.
* cfips- Um identificador exclusivo para cada condado usando o Sistema Federal de Processamento de Informações. Os dois primeiros dígitos correspondem ao código FIPS do estado, enquanto os 3 seguintes representam o município.
* county_name- O nome escrito do município.
* state_name- O nome do estado.
* first_day_of_month- A data do primeiro dia do mês.
microbusiness_density- Microempresas por 100 maiores de 18 anos no concelho em causa. Esta é a variável de destino. Os números populacionais usados ​​para calcular a densidade estão com um atraso de dois anos devido ao ritmo de atualização fornecido pelo US Census Bureau, que fornece os dados populacionais subjacentes anualmente. Os números da densidade de 2021 são calculados usando os números da população de 2019, etc.
active- Contagem bruta de microempresas no concelho. Não fornecido para o conjunto de teste.
sample_submission.csv Um envio de amostra válido. Este arquivo permanecerá inalterado durante toda a competição.

* row_id- Um código de ID para a linha.
* microbusiness_density- A variável de destino.

**test.csv**
 Metadados para as linhas de envio. Este arquivo permanecerá inalterado durante toda a competição.

* row_id- Um código de ID para a linha.
* cfips- Um identificador exclusivo para cada condado usando o Sistema Federal de Processamento de Informações. Os dois primeiros dígitos correspondem ao código FIPS do estado, enquanto os 3 seguintes representam o município.
* first_day_of_month- A data do primeiro dia do mês.
revelou_test.csv Durante o período de envio, apenas o mês mais recente de dados será usado para a tabela de classificação pública. Qualquer conjunto de dados de teste mais antigo que isso será publicado em revelou_test.csv , seguindo de perto o ciclo normal de liberação de dados para o relatório de microempresa. Esperamos publicar uma cópia de revelou_test.csv em meados de fevereiro. O esquema deste arquivo corresponderá ao train.csv .

**census_starter.csv** Exemplos de colunas úteis da American Community Survey (ACS) do Census Bureau em data.census.gov . Os campos de porcentagem foram derivados das contagens brutas fornecidas pelo ACS. Todos os campos têm um atraso de dois anos para corresponder às informações disponíveis no momento em que uma determinada atualização de dados de microempresa foi publicada.

* pct_bb_[year]- A percentagem de agregados familiares no concelho com acesso a qualquer tipo de banda larga. Derivado da tabela ACS B28002: PRESENÇA E TIPOS DE ASSINATURAS DE INTERNET EM DOMICÍLIOS.
* cfips- O código CFIPS.
pct_college_[year]- A porcentagem da população no condado com mais de 25 anos de idade com um diploma universitário de 4 anos. Derivado da tabela ACS S1501: CONCLUSÃO ESCOLARINGOLÓGICA.
* pct_foreign_born_[year]- A porcentagem da população do condado nascida fora dos Estados Unidos. Derivado da tabela ACS DP02: CARACTERÍSTICAS SOCIAIS SELECIONADAS NOS ESTADOS UNIDOS.
* pct_it_workers_[year]- A porcentagem da força de trabalho no condado empregada em indústrias relacionadas à informação. Derivado da tabela ACS S2405: INDÚSTRIA POR OCUPAÇÃO PARA A POPULAÇÃO CIVIL EMPREGADA DE 16 ANOS OU MAIS.
* median_hh_inc_[year]- A renda familiar média no município. Derivado da tabela ACS S1901: RENDA NOS ÚLTIMOS 12 MESES (EM DÓLARES AJUSTADOS À INFLAÇÃO DE 2021).

## Importando Bibliotecas

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import datetime as dt
from datetime import date, time, datetime, timedelta
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

## Base de Dados Godaddy Microbusiness

In [40]:
census_business = pd.read_csv('/content/drive/MyDrive/godaddy-microbusiness-density-forecasting/census_starter.csv', sep=',')
sample_business = pd.read_csv('/content/drive/MyDrive/godaddy-microbusiness-density-forecasting/sample_submission.csv', sep=',')
test_business = pd.read_csv('/content/drive/MyDrive/godaddy-microbusiness-density-forecasting/test.csv', sep=',')
train_business = pd.read_csv('/content/drive/MyDrive/godaddy-microbusiness-density-forecasting/train.csv', sep=',')

In [41]:
train_business.head()

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243


In [42]:
#simple statistical analysis
train_business.describe()

,cfips,microbusiness_density,active
count,122265.000000,122265.000000,1.222650e+05
mean,30376.037640,3.817671,6.442858e+03
std,15143.508721,4.991087,3.304001e+04
min,1001.000000,0.000000,0.000000e+00
25%,18177.000000,1.639344,1.450000e+02
50%,29173.000000,2.586543,4.880000e+02
75%,45077.000000,4.519231,2.124000e+03
max,56045.000000,284.340030,1.167744e+06


In [43]:
#Creating year, month and day columns
train_business[['year', 'month', 'day']] = train_business['first_day_of_month'].str.split("-", expand = True).astype(int)
test_business[['year', 'month', 'day']] = train_business['first_day_of_month'].str.split("-", expand = True).astype(int)

In [44]:
train_business

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,year,month,day
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,2019,8,1
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198,2019,9,1
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269,2019,10,1
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243,2019,11,1
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243,2019,12,1
...,...,...,...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101,2022,6,1
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101,2022,7,1
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100,2022,8,1
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100,2022,9,1


In [45]:
train_business.drop(['first_day_of_month'],axis=1, inplace=True)

In [46]:
train_business

,row_id,cfips,county,state,microbusiness_density,active,year,month,day
0,1001_2019-08-01,1001,Autauga County,Alabama,3.007682,1249,2019,8,1
1,1001_2019-09-01,1001,Autauga County,Alabama,2.884870,1198,2019,9,1
2,1001_2019-10-01,1001,Autauga County,Alabama,3.055843,1269,2019,10,1
3,1001_2019-11-01,1001,Autauga County,Alabama,2.993233,1243,2019,11,1
4,1001_2019-12-01,1001,Autauga County,Alabama,2.993233,1243,2019,12,1
...,...,...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,1.803249,101,2022,6,1
122261,56045_2022-07-01,56045,Weston County,Wyoming,1.803249,101,2022,7,1
122262,56045_2022-08-01,56045,Weston County,Wyoming,1.785395,100,2022,8,1
122263,56045_2022-09-01,56045,Weston County,Wyoming,1.785395,100,2022,9,1


In [47]:
#checking missing data
train_business.isna().sum()

row_id                   0
cfips                    0
county                   0
state                    0
microbusiness_density    0
active                   0
year                     0
month                    0
day                      0
dtype: int64

In [49]:
train_business['state'].value_counts()

Texas                   9906
Georgia                 6201
Virginia                5070
Kentucky                4680
Missouri                4485
Kansas                  4095
Illinois                3978
North Carolina          3900
Iowa                    3861
Tennessee               3705
Nebraska                3588
Indiana                 3588
Ohio                    3432
Minnesota               3393
Michigan                3237
Mississippi             3198
Oklahoma                3003
Arkansas                2925
Wisconsin               2808
Pennsylvania            2613
Alabama                 2613
Florida                 2613
South Dakota            2535
Colorado                2496
Louisiana               2496
New York                2418
California              2262
Montana                 2184
West Virginia           2145
North Dakota            2067
South Carolina          1794
Idaho                   1716
Washington              1521
Oregon                  1404
New Mexico    

In [50]:
df_texas = train_business[train_business['state'] == 'Texas']
df_texas

,row_id,cfips,county,state,microbusiness_density,active,year,month,day
98241,48001_2019-08-01,48001,Anderson County,Texas,1.649001,767,2019,8,1
98242,48001_2019-09-01,48001,Anderson County,Texas,1.627502,757,2019,9,1
98243,48001_2019-10-01,48001,Anderson County,Texas,1.627502,757,2019,10,1
98244,48001_2019-11-01,48001,Anderson County,Texas,1.653301,769,2019,11,1
98245,48001_2019-12-01,48001,Anderson County,Texas,1.631802,759,2019,12,1
...,...,...,...,...,...,...,...,...,...
108142,48507_2022-06-01,48507,Zavala County,Texas,0.413418,35,2022,6,1
108143,48507_2022-07-01,48507,Zavala County,Texas,0.425230,36,2022,7,1
108144,48507_2022-08-01,48507,Zavala County,Texas,0.413418,35,2022,8,1
108145,48507_2022-09-01,48507,Zavala County,Texas,0.413418,35,2022,9,1


In [52]:
fig = px.line(df_texas, x='county', y='active', title='Texas State Analysis Chart')
fig.show()